In [39]:
from typing import List, Dict, Any, Optional, Callable
from abc import ABC, abstractmethod
import json
from dataclasses import dataclass
from dotenv import load_dotenv
load_dotenv()

@dataclass
class Message:
    role: str
    content: str
    metadata: Optional[Dict[str, Any]] = None

class LLMProvider(ABC):
    @abstractmethod
    def generate(self, prompt: str, **kwargs) -> str:
        pass

In [106]:
class OpenAIProvider(LLMProvider):
    def __init__(self, model: str = "gpt-4o-mini"):
        from openai import OpenAI
        self.client = OpenAI()
        self.model = model

    def generate(self, prompt: List[Message], **kwargs) -> tuple[str | dict, bool]:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=prompt,
            **kwargs
        )
        if response.choices[0].message.tool_calls is None:
            return response.choices[0].message.content, False
        else:
            return response.choices[0].message, True
       

In [107]:
llm = OpenAIProvider()

In [64]:
import os
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=f"{os.getenv('TAVILY_API_KEY')}")

def search_tavily(query):
    response = tavily_client.search(
        query=query
    )
    return response['results']

In [65]:
tools = [{
    "type": "function",
    "function": {
        "name": "search_tavily",
        "description": "Search the web for a given query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "query to search the web for"
                }
            },
            "required": [
                "query"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [66]:
llm.generate("search on 'climate crisis'", tools=tools) 

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_iUrHCbMUlIRLuVByAIvY1riv', function=Function(arguments='{"query":"climate crisis"}', name='search_tavily'), type='function')])

In [67]:
class Tool:
    def __init__(
        self,
        description: List[Dict[str, Any]],
        function: Callable
    ):
        self.description = description
        self.function = function
   
        

In [115]:
class Agent:
    def __init__(
        self,
        llm: LLMProvider,
        system_prompt: str,
        tool: Tool = None
    ):
        self.llm = llm
        self.system_prompt = system_prompt
        self.messages = [{"role": "system", "content": self.system_prompt}]
        self.tool = tool

    def generate(self, prompt: str, **kwargs) -> str:
        self.messages.append({"role": "user", "content": prompt})

        if self.tool:
            completion, is_tool_call = self.llm.generate(
                prompt=self.messages,
                tools=self.tool.description
            )
            
            if is_tool_call:
                tool_call = completion.tool_calls[0]
                args = json.loads(tool_call.function.arguments)

                result = self.tool.function(args["query"])
                self.messages.append(completion)
                self.messages.append({                             
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
                
                completion_2, _ = self.llm.generate(
                    prompt=self.messages,
                )

                return completion_2
            else:
                return completion
        else:
            completion = self.llm.generate(
                prompt=self.messages                
            )
            return completion
        

In [95]:
search_tool = Tool(tools, search_tavily)

In [119]:
search_agent = Agent(
    llm=llm, 
    system_prompt="사용자가 요청한 내용에 대한 팩트체크를 위한 인터넷 검색을 해주는 도우미입니다.",
    tool=search_tool
)

In [120]:
fact_check_agent = Agent(
    llm=llm, 
    system_prompt="주어진 내용을 요약하고 팩트체크를 해주는 도우미입니다."    
)

In [123]:
searched_result = search_agent.generate("지구는 평평하다")

In [124]:
searched_result

'지구가 평평하다는 주장은 과학적으로 근거가 없습니다. 현대 과학은 지구가 둥글다는 것을 명확한 증거로 입증하였습니다. 평평한 지구론자들은 여러 가지 주장과 이론을 제기하지만, 이들은 주로 음모론적 성격을 띠고 있으며 과학적 사실과 반대되는 내용입니다.\n\n예를 들어, 지구의 곡률, 위성 사진, 항공기 비행 경로와 같은 여러 과학적 데이터는 지구가 둥글다는 것을 지지합니다. NASA의 이미지와 우주 탐사기술 등도 지구의 구형을 나타냅니다. 평면 지구론은 현대 과학의 이해와 부합하지 않으며, 이를 믿는 사람들은 종종 잘못된 정보와 음모론에 기반해 설명을 시도합니다.\n\n더 많은 정보는 [여기](https://blog.naver.com/PostView.naver?blogId=bab_bab_2&logNo=223729348974)에서 확인할 수 있습니다.'

In [125]:
fact_check_agent.generate(searched_result)

('주어진 내용은 지구가 평평하다는 주장에 대한 반론입니다. 요약하자면, 현대 과학은 지구가 둥글다는 것을 여러 가지 증거를 통해 입증하고 있으며, 평평한 지구론은 과학적 사실과 반대되는 음모론적 주장을 포함하고 있다고 설명합니다. 증거로는 지구의 곡률, 위성 사진, 항공기 비행 경로 등을 들고 있습니다.\n\n이와 관련하여 사실 확인을 해보면, 지구가 둥글다는 것은 과학적 합의로, 많은 실증적 증거에 의해 지지되고 있습니다. NASA의 이미지나 기타 우주 탐사 데이터는 지구의 구형성을 뒷받침하며, 평면 지구론은 현대 과학의 이해와 명백히 상반되는 주장입니다. 따라서, 평면 지구론이 과학적 근거가 없다는 주장은 사실입니다.',
 False)